In [1]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)

In [2]:
from sklearn.linear_model import LinearRegression as LR
from numba import jit

In [3]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
#days_a, days_b = set(days[:n_days // 2]), set(days[n_days // 2:])
tr, cv, ep = df.Day.isin(days_a), df.Day.isin(days_b), df.Day.isin(days_c)
#tr, cv = df.Day.isin(days_a), df.Day.isin(days_b)

In [4]:
duse = dff
rawfeats = x1feats

In [5]:
def_matrices(globals(), duse, 10000)

In [6]:
for xname in rawfeats:
    X = duse.pivot('Day', 'Stock', xname)
    Xa = X.fillna(0)
    Xm = X.fillna(X.mean())
    Xpct = X.pct(positive=True)
    Xpcta = Xpct.fillna(0)
    Xpctm = Xpct.fillna(0.5)
    for ext in ['', 'a', 'm', 'pct', 'pcta', 'pctm']:
        globals()[xname.upper() + ext] = locals()['X' + ext]

In [7]:
tr, cv, ep = Y.index.isin(days_a), Y.index.isin(days_b), Y.index.isin(days_c)

In [8]:
def mse(Y_, st):
    return (W[st] * (Y_ - Y[st]) ** 2).sum().sum() / W[st].sum().sum()

def rmse(Y_, st):
    return np.sqrt(mse(Y_, st))

def prss(Y_, st):
    return mse(Y_, st) / mse(0, st)

def lbss(Y_, st):
    return prss(Y_, st) * 0.500944

def metric(Y_, st):
    print('(' + ', '.join('{:.6f}'.format(round(func(Y_, st), 6)) for func in [rmse, prss, lbss]) + ')')

In [9]:
Xa = pd.concat({x: globals()[x.upper() + 'a'] for x in rawfeats}, axis=1).swaplevel(axis=1).sort_index(axis=1)
Xpcta = pd.concat({x: globals()[x.upper() + 'pcta'] for x in rawfeats}, axis=1).swaplevel(axis=1).sort_index(axis=1)

In [10]:
Xuse = Xpcta #.loc[:, (ss, 'x4')]

In [66]:
def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in range(N_STOCK):
        x, y, w = X[k], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=True)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x, y, w)
        lr_list += [lr]
    return lr_list

lr_list = do_lr(Xuse[tr], Ya[tr], Wa[tr])

##### small

In [51]:
Yx = np.stack([(lr.predict(Xuse[k]) if lr is not None else np.zeros(Y.shape[0]))
               for lr, k in zip(lr_list, range(N_STOCK))], axis=1)
Yx = Yx + Y * 0

In [52]:
st = cv
Yxm = Yx.where(Y[tr].count() > 50, Y[tr].mean().fillna(0), axis=1)

##### big

In [20]:
bigX = Xpcta.stack(level=0)
bigY = Ya.stack(level=0)
bigW = Wa.stack(level=0)

In [24]:
lr = LR()
lr = lr.fit(bigX, bigY, bigW)

In [28]:
Yx = np.stack([(lr.predict(Xpcta[k]) if lr is not None else np.zeros(Y.shape[0]))
               for k in range(N_STOCK)], axis=1)
Yxm = Yx + Y * 0

### metric

In [53]:
metric(Yxm[tr], tr)

(2.869601, 0.748053, 0.374732)


In [54]:
metric(Yxm[cv], cv)

(2.927475, 0.795920, 0.398711)


In [21]:
metric(Y[tr].wmean(W[tr]), cv)

(2.951820, 0.809212, 0.405370)


In [22]:
metric(Y[tr].mean(), cv)

(2.961588, 0.814577, 0.408057)


In [19]:
metric(0, cv)

(3.281396, 1.000000, 0.500944)


In [18]:
metric(Y[cv]*.225, cv)

(2.543082, 0.600625, 0.300879)


### stop

In [ ]:
raise Stop

In [ ]:
k = 2
plt.scatter(Xuse[k], Ya[k]);
plt.plot(Xuse[k], Yx[k]);
plt.xlim(-0.0001, 0.0004);
plt.ylim(-0.0002, 0.0001);

# legit alg

In [ ]:
def alg():
    lastminguy = None
    init(scores)
    init(heap)
    while True:
        minscore, minguy = heap.pop()[1]
        for f in features - minguy:
            newguy = minguy + {f}
            if newguy not in scores:
                score = eval(newguy)
                scores[newguy] = score
                heap += (score, newguy)
        if minscore <= heap[0][0]:
            break

In [108]:
from numba.types import List, int64, float64, UniTuple

In [169]:
from heapq import *

@jit(nopython=True)
def _lr_ready(x):
    return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

@jit(nopython=True)
def _lr_fit(x, y, w):
    x = _lr_ready(x)
    wsqrt = w ** 0.5
    thing = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * x, wsqrt * y)
    return thing[0]

@jit(nopython=True)
def _lr_predict(x, beta):
    x = _lr_ready(x)
    return x @ beta

@jit(nopython=True)
def _lr_rss(y, y_, w):
    return ((y_ - y) ** 2 * w).sum()

from numba.types import List, int64, float64

#@jit(UniTuple(float64, 2)(float64[:,:], float64[:,:], float64[:,:],
#                          float64[:,:], float64[:,:], float64[:,:],
#                          float64[:,:], float64[:,:], float64[:,:], int64, int64, List(int32)), nopython=True)
@jit(nopython=True)
def _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g * k : (g + 1) * k][:, subk], ytr[:, g], wtr[:, g],
            xcv[:, g * k : (g + 1) * k][:, subk], ycv[:, g], wcv[:, g],
            xep[:, g * k : (g + 1) * k][:, subk], yep[:, g], wep[:, g])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        ycvg_ = _lr_predict(xcvg, beta)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, beta)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

def alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    m = xtr.columns.get_level_values(0).nunique()
    k = xtr.columns.get_level_values(1).nunique()
    features = set(range(k))
    xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep = (
        xtr.values, ytr.values, wtr.values, xcv.values, ycv.values, wcv.values, xep.values, yep.values, wep.values)
    #
    baseline = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, np.array([], dtype=np.int64))
    scores = {(): baseline}
    heap = [(baseline[0], ())]
    while True:
        minscore0, minfeats = heappop(heap)
        print(minscore0 / baseline[0], scores[minfeats][1] / baseline[1], len(minfeats))
        print(minfeats)
        print()
        for f in features - set(minfeats):
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep,
                                         m, k, np.array(newfeats, dtype=np.int64))
                scores[newfeats] = newscore
                heappush(heap, (newscore[0], newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        if not heap or minscore0 <= heap[0][0]:
            break
    return

In [170]:
Xtest = Xuse
Ytest = Ya
Wtest = Wa
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep])

1.0 1.0 0
()

0.9021508609439227 0.8921863375671456 1
(0,)

0.8769865287219386 0.8616810220246435 2
(0, 4)

0.8454299374523643 0.8299923493265597 3
(0, 3, 4)



### old implementations

In [222]:
def alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    m, k = len(groups), len(features)
    #
    #
    #@jit(nopython=True)
    def evaluate(feats):
        cvs = []
        eps = []
        for g in groups:
            beta = lr_fit(xtr.loc[:, (g, feats)].values, ytr.loc[:, g].values, wtr.loc[:, g].values)
            y_ = lr_predict(xcv.loc[:, (g, feats)].values, beta)
            cvs += [lr_rss(ycv.loc[:, g].values, y_, wcv.loc[:, g].values)]
            y_ = lr_predict(xep.loc[:, (g, feats)].values, beta)
            eps += [lr_rss(yep.loc[:, g], y_, wep.loc[:, g].values)]
        return sum(cvs), sum(eps)
    #
    baseline, baselinetest = evaluate(())
    scores, scorestest = {(): baseline}, {(): baselinetest}
    heap = [(baseline, ())]
    #
    while True:
        minscore, minfeats = heappop(heap)
        print(minscore / baseline, scorestest[minfeats] / baselinetest, len(minfeats))
        print(minfeats)
        print()
        for f in features - set(minfeats):
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore, newscoretest = evaluate(newfeats)
                scores[newfeats], scorestest[newfeats] = newscore, newscoretest
                heappush(heap, (newscore, newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        if not heap or minscore <= heap[0][0]:
            break
    return

In [18]:
@jit(nopython=True)
def _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g*k:(g+1)*k], ytr[:, g*k:(g+1)*k], wtr[:, g*k:(g+1)*k],
            xcv[:, g*k:(g+1)*k], ycv[:, g*k:(g+1)*k], wcv[:, g*k:(g+1)*k],
            xep[:, g*k:(g+1)*k], yep[:, g*k:(g+1)*k], wep[:, g*k:(g+1)*k])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        ycvg_ = _lr_predict(xcvg, ycvg, wcvg)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, yepg, wepg)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

@jit(nopython=True)
def _alg_help_featsin(scored, feats):
    np.

@jit(nopython=True)
def _alg_help_featsadd(scored, feats):
    pass

@jit(nopython=True)
def _alg_main(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, maxnumfeats, m, k):
    baseline, baselinetest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, ())
    scored = np.zeros((k, 1), np.bool)
    #scoreslist = []
    heap = {(baseline, ())}
    minscore = np.inf
    while True:
        newminscore, newminfeats = min(heap)
        heap.remove((newminscore, newminfeats))
        if minscore <= newminscore:
            break
        minscore, minfeats = newminscore, newminfeats
        if len(minfeats) >= maxnumfeats:
            break
        minfeatsset = set(minfeats)
        for f in [f for f in range(k) if f not in minfeatsset]:
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scored:
                newscore, newscoretest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, newfeats)
                heap.add((newscore, newfeats))
                scored.add(newfeats)
                #scoreslist.append((newscore, newscoretest, newfeats))
        if not heap:
            break
    return scores[minfeats], scorestest[minfeats], minfeats

def alg_numba(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    n, m, k = len(xtr), len(groups), len(features)
    cvs, eps, feats = _alg_main(
        xtr.values, ytr.values, wtr.values, xcv.values, ycv.values, wcv.values, xep.values, yep.values, wep.values,
        40, m, k)
    return cvs, eps, features[feats]

In [19]:
Xtest = Xuse
Ytest = Ya
Wtest = Wa
alg_numba(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep])

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'tuple': cannot determine Numba type of <class 'type'>
File "<ipython-input-18-d2daf27047d8>", line 33

In [49]:
numba.typeof()

ValueError: cannot determine Numba type of <class 'type'>

In [65]:
@jit(nopython=True)
def derp(a):
    return a[0]

In [79]:
for i in range(1, 10000):
    a = list(range(i))
    derp(a)